# Preprocessing: Cropping

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile as tiff
from tifffile import imsave
from tifffile import TiffWriter
from PIL import Image
import natsort
from ipywidgets import interact, interactive, fixed, interact_manual


### First, read an example image file

In [8]:
# Find the data

data_folder = '/users/charles.fieseler/shared_projects/ForCharlie/27082020_freelyimmobilized_trial2/'
data_file = 'ZIM2051_trial_21_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'
# data_file = 'ZIM2051_trial_11_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'

full_fname = os.path.join(data_folder, data_file)


In [9]:
# Actually read the data in
im = tiff.imread(full_fname)

In [20]:
def myplt(i):
    plt.figure(figsize=[15,15])
    plt.imshow(im[i,...])
    plt.title(f"Slice {i}")
    plt.colorbar()
interact(myplt, i=(0, 33))

interactive(children=(IntSlider(value=16, description='i', max=33), Output()), _dom_classes=('widget-interact'…

<function __main__.myplt(i)>

### Determine the crop via a GUI

Uses this package:
https://openbits.app/posts/python-interactive-cropping/

pip install interactivecrop